# 1. Data Generator
- Raw Data를 읽어옴
- 여기서 만들어진 데이터는 모델의 입력으로 들어감

In [1]:
import os
import numpy as np
import librosa
from tensorflow.keras.utils import Sequence

In [2]:
class RawForVAEGenerator(Sequence):
    def __init__(self, source, wav_dir, files, sourNum='s1', batch_size=10, shuffle=True):
        self.source = source
        self.wav_dir = wav_dir
        self.files = files
        self.sourNum = sourNum
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
        self.sample_rate = 8000
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.source))
        
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __audioread__(self, path, offset=0.0, duration=None, sample_rate=16000):
        signal = librosa.load(path, sr=self.sample_rate, mono=False, offset=offset, duration=duration)

        return signal[0]
    
    def __padding__(self, data):
        n_batch = len(data)
        max_len = max([d.shape[0] for d in data])
        extrapadding = int(np.ceil(max_len / self.sample_rate) * self.sample_rate)
        pad = np.zeros((n_batch, extrapadding))
        
        for i in range(n_batch):
            pad[i, :data[i].shape[0]] = data[i]
        
        return np.expand_dims(pad, -1)
        
    def __data_generation__(self, source_list):
        wav_list = []
        for name in source_list:
            name = name.strip('\n')
            
            s_wav_name = self.wav_dir + self.files + '/' + self.sourNum + '/' + name
            
            # ------- AUDIO READ -------
            s_wav = (self.__audioread__(s_wav_name,  offset=0.0, duration=None, sample_rate=self.sample_rate))
            # --------------------------
            
            # ------- PADDING -------
#             pad_len = max(len(samples1),len(samples2))
#             pad_s1 = np.concatenate([s1_wav, np.zeros([pad_len - len(s1_wav)])])
            
#             extrapadding = ceil(len(pad_s1) / sample_rate) * sample_rate - len(pad_s1)
#             pad_s1 = np.concatenate([pad_s1, np.zeros([extrapadding - len(pad_s1)])])
#             pad_s2 = np.concatenate([s2_wav, np.zeros([extrapadding - len(s2_wav)])])
            # -----------------------
            
            wav_list.append(s_wav)
        
        return wav_list, wav_list, source_list
            
    
    def __len__(self):
        return int(np.floor(len(self.source) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        source_list = [self.source[k] for k in indexes]
        
        if self.files is not 'tt':
            sour, labels, _ = self.__data_generation__(source_list)
            
            # Get Lengths(K value of each batch)
            lengths = np.array([m.shape[0] for m in sour])
            exp = np.expand_dims(lengths, 1)
            exp = np.expand_dims(exp, -1) # [Batch, 1, 1] (length)
            
            # Padding
            sour_pad = self.__padding__(sour) # [Batch, Time_step, Dimension(=1)]
            label_pad = self.__padding__(labels) # [Batch, Time_step, Dimension(=1)]
            
            return sour_pad, np.concatenate([label_pad, exp], axis=1)
        else:
            sour, labels, name = self.__data_generation__(source_list)
            
            # Get Lengths(K value of each batch)
            lengths = np.array([m.shape[0] for m in sour])
            exp = np.expand_dims(lengths, 1)
            exp = np.expand_dims(exp, -1) # [Batch, 1, 1] (length)
            
            # Padding
            sour_pad = self.__padding__(sour) # [Batch, Time_step, Dimension(=1)]
            
            return sour_pad, exp, name

## Data를 어떻게 읽는지에 대한 부분

In [3]:
WAV_DIR = './mycode/wsj0_2mix/use_this/'
LIST_DIR = './mycode/wsj0_2mix/use_this/lists/'

In [4]:
# Directory List file create

wav_dir = WAV_DIR
output_lst = LIST_DIR

for folder in ['tr', 'cv', 'tt']:
    wav_files = os.listdir(wav_dir + folder + '/mix')
    output_lst_files = output_lst + folder + '_wav.lst'
    with open(output_lst_files, 'w') as f:
        for file in wav_files:
            f.write(file + "\n")

print("Generate wav file to .lst done!")

Generate wav file to .lst done!


In [5]:
batch_size = 2

train_dataset = 0
valid_dataset = 0
test_dataset = 0

name_list = []
for files in ['tr', 'cv', 'tt']:
    # --- Lead lst file ---""
    output_lst_files = LIST_DIR + files + '_wav.lst'
    fid = open(output_lst_files, 'r')
    lines = fid.readlines()
    fid.close()
    # ---------------------
    
    if files == 'tr':
        train_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', batch_size)
    elif files == 'cv':
        valid_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', batch_size)
    else:
        test_batch = 1
        test_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', test_batch)

# 2. Building VQ-VAE model with Gumbel Softmax

In [6]:
import threading
from scipy.io.wavfile import write as wav_write
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from tensorflow.keras import backend as Kb
import numpy as np
import pandas as pd
from importlib import reload
import time
from tensorflow.keras.models import Model, Sequential, load_model

In [7]:
def mkdir_p(path):
    """ Creates a path recursively without throwing an error if it already exists
    :param path: path to create
    :return: None
    """
    if not os.path.exists(path):
        os.makedirs(path)

In [8]:
mkdir_p('./CKPT/') # model check point 폴더 만드는 코드
filepath = "./CKPT/CKP_ep_{epoch:d}__loss_{val_loss:.5f}_.h5"

In [9]:
initial_learning_rate = 0.001

# learning rate를 점점 줄이는 부분
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

# validation loss에 대해서 좋은 것만 저장됨
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min'
)

# early stop 하는 부분인데, validation loss에 대해서 제일 좋은 모델이 저장됨
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=50, restore_best_weights=True
)

In [10]:
class GumbelSoftmax(layers.Layer):
    def __init__(self, temperature=1.0, hard=False, name = 'gumbel_softmax',**kwargs):
        super(GumbelSoftmax, self).__init__(name=name, **kwargs)
        
        self.temperature = temperature
        self.hard = hard
    
    def sample_gumbel(self, shape, eps=1e-20): 
        """Sample from Gumbel(0, 1)"""
        U = tf.random.uniform(shape,minval=0,maxval=1)
        
        return -tf.math.log(-tf.math.log(U + eps) + eps)

    def gumbel_softmax_sample(self, logits, temperature): 
        """ Draw a sample from the Gumbel-Softmax distribution"""
        y = logits + self.sample_gumbel(tf.shape(logits))
        
        return tf.nn.softmax(y / temperature)

    def call(self, inputs):
        y = self.gumbel_softmax_sample(inputs, self.temperature)
        
        if self.hard:
            y_hard = tf.cast(tf.equal(y, tf.math.reduce_max(y, 2, keepdims=True)), y.dtype)
            y = tf.stop_gradient(y_hard - y) + y
        
        return y


class Encoder(layers.Layer):
    def __init__(self, latent_dim, name = 'encoder',**kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        
        self.conv1d_1 = layers.Conv1D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu')
        self.conv1d_2 = layers.Conv1D(filters=128, kernel_size=3, strides=2, padding='same', activation='relu')
        self.conv1d_3 = layers.Conv1D(filters=256, kernel_size=3, strides=2, padding='same', activation='relu')
        self.logit = layers.Conv1D(filters=latent_dim, kernel_size=3, strides=2, activation='relu', padding='same')
    
    def call(self, inputs):
        x = self.conv1d_1(inputs)
        x = self.conv1d_2(x)
        x = self.conv1d_3(x)
        logit = self.logit(x)
        
        return logit


class Decoder(layers.Layer):
    def __init__(self, latent_dim, name = 'decoder',**kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        
        self.trans_conv1d_1 = layers.Conv1DTranspose(filters=256, kernel_size=3, strides=2, activation='relu', padding='same')
        self.trans_conv1d_2 = layers.Conv1DTranspose(filters=128, kernel_size=3, strides=2, activation='relu', padding='same')
        self.trans_conv1d_3 = layers.Conv1DTranspose(filters=64, kernel_size=3, strides=2, activation='relu', padding='same')
        self.logit = layers.Conv1DTranspose(filters=1, kernel_size=3, strides=2, padding='same', activation=None)
    
    def call(self, inputs):
        x = self.trans_conv1d_1(inputs)
        x = self.trans_conv1d_2(x)
        x = self.trans_conv1d_3(x)
        logit = self.logit(x)
        
        return logit

In [11]:
# Custom Metric Si-sdr

class SiSdr(keras.metrics.Metric):
    def __init__(self, name="Si-sdr", **kwargs):
        super(SiSdr, self).__init__(name=name, **kwargs)
        self.sdr = self.add_weight(name="sdr", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        ori_length = tf.shape(y_true)[1]
        
        # Label & Length divide
        labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 1]
        lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]
        
        # Check sequence length
        batch_size = tf.shape(labels)[0]
        label_size = tf.shape(labels)[1]
        pred_size = tf.shape(y_pred)[1]
        feature_size = tf.shape(labels)[-1]
        
        # Change sequence length
        if label_size < pred_size:
            y_pred = tf.slice(y_pred, [0, 0, 0], [-1, label_size, -1])
        elif label_size > pred_size:
            labels = tf.slice(labels, [0, 0, 0], [-1, pred_size, -1])

        # SI-SDR
        target = tf.linalg.matmul(y_pred, labels, transpose_a=True) * labels / tf.expand_dims(tf.experimental.numpy.square(tf.norm(labels, axis=1)), axis=-1)
        noise = y_pred - target
        values = 10 * tf.experimental.numpy.log10(tf.experimental.numpy.square(tf.norm(target, axis=1)) / tf.experimental.numpy.square(tf.norm(noise, axis=1)))
        
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.sdr.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.sdr

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.sdr.assign(0.0)

In [12]:
# Custom loss

# Custom mse
def custom_mse(y_true, y_pred):
    ori_length = tf.shape(y_true)[1]

    # Label & Length divide
    labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 129]
    lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]

    loss = tf.reduce_sum(tf.pow(y_pred - labels, 2))

    return loss


# Custom si-sdr loss
def custom_sisdr_loss(y_true, y_pred):
    ori_length = tf.shape(y_true)[1]

    # Label & Length divide
    labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 1]
    lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]

    target = tf.linalg.matmul(y_pred, labels, transpose_a=True) * labels / tf.expand_dims(tf.experimental.numpy.square(tf.norm(labels, axis=1)), axis=-1)
    noise = y_pred - target
    si_sdr = 10 * tf.experimental.numpy.log10(tf.experimental.numpy.square(tf.norm(target, axis=1)) / tf.experimental.numpy.square(tf.norm(noise, axis=1)))
    si_sdr = tf.reduce_mean(si_sdr) * -1

    return si_sdr

In [13]:
class Vq_vae(keras.Model):
    def __init__(self, latent_dim, gumbel_hard=False, name='vqvae', **kwargs):
        super(Vq_vae, self).__init__(name=name, **kwargs)
        
        self.latent_dim = latent_dim
        self.softmax = layers.Softmax(-1)
        
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)
        self.gumbel = GumbelSoftmax(hard=gumbel_hard)
        
    def call(self, inputs, load=False):
        if load:
            inputs = layers.Input(shape=(None, 1))
        
        
        encode = self.encoder(inputs)
        gumbel = self.gumbel(encode)
        decode = self.decoder(gumbel)
        
        # ------------------ KL loss ------------------
        qy = self.softmax(encode)
        log_qy = tf.math.log(qy + 1e-20)
        log_uniform = qy * (log_qy - tf.math.log(1.0 / self.latent_dim))
        kl_loss = tf.reduce_mean(log_uniform)
        # ---------------------------------------------
        
        self.add_loss(kl_loss)
        
        return decode

In [14]:
latent_size = 512
epoch = 300
BATCH_SIZE = 2

strategy = tf.distribute.MirroredStrategy(['cpu:0'])
print('장치의 수: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    model_path = './CKPT/CKP_ep_30__loss_158.95885_.h5'
    
    loss_fun = custom_mse
#     loss_fun = custom_sisdr_loss
    
    vq_vae = Vq_vae(latent_size, gumbel_hard=True)

    optimizer = keras.optimizers.Adam(learning_rate=1e-3)
    vq_vae.compile(optimizer, loss=loss_fun, metrics=[SiSdr()])
    
    vq_vae(0, True)
    vq_vae.summary()
    
    # 사용 안할 때는 load_model 주석 처리 하자
#     vq_vae.load_weights(model_path)
    # ----------------------------------------
    
    tf.executing_eagerly()

history = vq_vae.fit(
    train_dataset,
    epochs=epoch,
    validation_data=valid_dataset,
    shuffle=True,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
장치의 수: 1
Model: "vqvae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
softmax (Softmax)            (None, None, 512)         0         
_________________________________________________________________
encoder (Encoder)            (None, None, 512)         517248    
_________________________________________________________________
decoder (Decoder)            (None, None, 1)           516737    
_________________________________________________________________
gumbel_softmax (GumbelSoftma (None, None, 512)         0         
Total params: 1,033,985
Trainable params: 1,033,985
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
2/2 [==============================] - ETA: 0s - loss: 634.7297 - Si-sdr: -260.8775

c:\users\moon\anaconda3\envs\study\lib\site-packages\tensorflow\python\keras\metrics.py:257: UserWarning: Metric SiSdr implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))


2/2 [==============================] - 5s 2s/step - loss: 634.7297 - Si-sdr: -260.8775 - val_loss: 632.0584 - val_Si-sdr: -222.9522

Epoch 00001: val_loss improved from inf to 632.05841, saving model to ./CKPT\CKP_ep_1__loss_632.05841_.h5
Epoch 2/300
2/2 [==============================] - 1s 804ms/step - loss: 631.5106 - Si-sdr: -244.2579 - val_loss: 629.4757 - val_Si-sdr: -243.2752

Epoch 00002: val_loss improved from 632.05841 to 629.47571, saving model to ./CKPT\CKP_ep_2__loss_629.47571_.h5
Epoch 3/300
2/2 [==============================] - 1s 763ms/step - loss: 628.7528 - Si-sdr: -219.0233 - val_loss: 628.1381 - val_Si-sdr: -202.9614

Epoch 00003: val_loss improved from 629.47571 to 628.13806, saving model to ./CKPT\CKP_ep_3__loss_628.13806_.h5
Epoch 4/300
2/2 [==============================] - 1s 664ms/step - loss: 628.3107 - Si-sdr: -222.8348 - val_loss: 628.1638 - val_Si-sdr: -250.7644

Epoch 00004: val_loss did not improve from 628.13806
Epoch 5/300
2/2 [=======================

2/2 [==============================] - 1s 650ms/step - loss: 328.7020 - Si-sdr: -1.3753 - val_loss: 323.0475 - val_Si-sdr: -0.7638

Epoch 00033: val_loss improved from 331.54022 to 323.04755, saving model to ./CKPT\CKP_ep_33__loss_323.04755_.h5
Epoch 34/300
2/2 [==============================] - 1s 644ms/step - loss: 323.2036 - Si-sdr: -0.6322 - val_loss: 309.1516 - val_Si-sdr: 0.7496

Epoch 00034: val_loss improved from 323.04755 to 309.15155, saving model to ./CKPT\CKP_ep_34__loss_309.15155_.h5
Epoch 35/300
2/2 [==============================] - 1s 641ms/step - loss: 308.9323 - Si-sdr: 0.7294 - val_loss: 294.5701 - val_Si-sdr: 2.2450

Epoch 00035: val_loss improved from 309.15155 to 294.57007, saving model to ./CKPT\CKP_ep_35__loss_294.57007_.h5
Epoch 36/300
2/2 [==============================] - 1s 608ms/step - loss: 295.4250 - Si-sdr: 2.2456 - val_loss: 288.5581 - val_Si-sdr: 2.9431

Epoch 00036: val_loss improved from 294.57007 to 288.55814, saving model to ./CKPT\CKP_ep_36__loss_

2/2 [==============================] - 1s 613ms/step - loss: 199.7366 - Si-sdr: 13.0513 - val_loss: 197.9937 - val_Si-sdr: 13.1789

Epoch 00066: val_loss did not improve from 196.45392
Epoch 67/300
2/2 [==============================] - 1s 720ms/step - loss: 195.0018 - Si-sdr: 13.4929 - val_loss: 197.5035 - val_Si-sdr: 13.3415

Epoch 00067: val_loss did not improve from 196.45392
Epoch 68/300
2/2 [==============================] - 1s 642ms/step - loss: 195.1840 - Si-sdr: 13.4990 - val_loss: 189.1712 - val_Si-sdr: 14.2989

Epoch 00068: val_loss improved from 196.45392 to 189.17120, saving model to ./CKPT\CKP_ep_68__loss_189.17120_.h5
Epoch 69/300
2/2 [==============================] - 1s 634ms/step - loss: 189.9528 - Si-sdr: 14.2065 - val_loss: 190.4040 - val_Si-sdr: 14.0972

Epoch 00069: val_loss did not improve from 189.17120
Epoch 70/300
2/2 [==============================] - 1s 641ms/step - loss: 191.4688 - Si-sdr: 13.9516 - val_loss: 191.7602 - val_Si-sdr: 13.7799

Epoch 00070: val


Epoch 00103: val_loss improved from 172.92836 to 171.39751, saving model to ./CKPT\CKP_ep_103__loss_171.39751_.h5
Epoch 104/300
2/2 [==============================] - 1s 687ms/step - loss: 172.4542 - Si-sdr: 16.3649 - val_loss: 170.8503 - val_Si-sdr: 16.7015

Epoch 00104: val_loss improved from 171.39751 to 170.85028, saving model to ./CKPT\CKP_ep_104__loss_170.85028_.h5
Epoch 105/300
2/2 [==============================] - 1s 718ms/step - loss: 172.1273 - Si-sdr: 16.5210 - val_loss: 174.5478 - val_Si-sdr: 16.1844

Epoch 00105: val_loss did not improve from 170.85028
Epoch 106/300
2/2 [==============================] - 1s 594ms/step - loss: 174.5616 - Si-sdr: 16.0929 - val_loss: 175.1502 - val_Si-sdr: 15.9056

Epoch 00106: val_loss did not improve from 170.85028
Epoch 107/300
2/2 [==============================] - 1s 607ms/step - loss: 172.8551 - Si-sdr: 16.4023 - val_loss: 176.1766 - val_Si-sdr: 15.8164

Epoch 00107: val_loss did not improve from 170.85028
Epoch 108/300
2/2 [=========


Epoch 00141: val_loss did not improve from 159.03656
Epoch 142/300
2/2 [==============================] - 1s 705ms/step - loss: 160.0375 - Si-sdr: 18.5107 - val_loss: 162.2784 - val_Si-sdr: 17.9225

Epoch 00142: val_loss did not improve from 159.03656
Epoch 143/300
2/2 [==============================] - 1s 710ms/step - loss: 160.9550 - Si-sdr: 17.9441 - val_loss: 165.4547 - val_Si-sdr: 17.5024

Epoch 00143: val_loss did not improve from 159.03656
Epoch 144/300
2/2 [==============================] - 1s 639ms/step - loss: 165.3210 - Si-sdr: 17.6751 - val_loss: 161.5985 - val_Si-sdr: 18.0631

Epoch 00144: val_loss did not improve from 159.03656
Epoch 145/300
2/2 [==============================] - 1s 711ms/step - loss: 162.5649 - Si-sdr: 17.9792 - val_loss: 158.6469 - val_Si-sdr: 18.4516

Epoch 00145: val_loss improved from 159.03656 to 158.64691, saving model to ./CKPT\CKP_ep_145__loss_158.64691_.h5
Epoch 146/300
2/2 [==============================] - 1s 620ms/step - loss: 157.5287 - Si-

2/2 [==============================] - 1s 729ms/step - loss: 155.2471 - Si-sdr: 18.9507 - val_loss: 159.3701 - val_Si-sdr: 18.2663

Epoch 00180: val_loss did not improve from 151.47202
Epoch 181/300
2/2 [==============================] - 1s 628ms/step - loss: 157.4455 - Si-sdr: 18.5917 - val_loss: 156.1230 - val_Si-sdr: 18.8511

Epoch 00181: val_loss did not improve from 151.47202
Epoch 182/300
2/2 [==============================] - 1s 689ms/step - loss: 155.1693 - Si-sdr: 18.9950 - val_loss: 155.3969 - val_Si-sdr: 18.8599

Epoch 00182: val_loss did not improve from 151.47202
Epoch 183/300
2/2 [==============================] - 1s 596ms/step - loss: 153.4191 - Si-sdr: 19.1476 - val_loss: 149.8477 - val_Si-sdr: 19.6674

Epoch 00183: val_loss improved from 151.47202 to 149.84772, saving model to ./CKPT\CKP_ep_183__loss_149.84772_.h5
Epoch 184/300
2/2 [==============================] - 1s 610ms/step - loss: 152.2370 - Si-sdr: 19.3061 - val_loss: 154.8158 - val_Si-sdr: 18.6971

Epoch 00184


Epoch 00219: val_loss did not improve from 145.75688
Epoch 220/300
2/2 [==============================] - 1s 631ms/step - loss: 150.4275 - Si-sdr: 19.5352 - val_loss: 144.9190 - val_Si-sdr: 20.3836

Epoch 00220: val_loss improved from 145.75688 to 144.91896, saving model to ./CKPT\CKP_ep_220__loss_144.91896_.h5
Epoch 221/300
2/2 [==============================] - 1s 715ms/step - loss: 144.5539 - Si-sdr: 20.4609 - val_loss: 143.6686 - val_Si-sdr: 20.5164

Epoch 00221: val_loss improved from 144.91896 to 143.66861, saving model to ./CKPT\CKP_ep_221__loss_143.66861_.h5
Epoch 222/300
2/2 [==============================] - 1s 705ms/step - loss: 144.4215 - Si-sdr: 20.2803 - val_loss: 151.3765 - val_Si-sdr: 19.2733

Epoch 00222: val_loss did not improve from 143.66861
Epoch 223/300
2/2 [==============================] - 1s 647ms/step - loss: 154.0011 - Si-sdr: 18.8763 - val_loss: 149.1919 - val_Si-sdr: 19.5616

Epoch 00223: val_loss did not improve from 143.66861
Epoch 224/300
2/2 [=========

2/2 [==============================] - 1s 617ms/step - loss: 147.2238 - Si-sdr: 19.9538 - val_loss: 146.4517 - val_Si-sdr: 19.8904

Epoch 00260: val_loss did not improve from 143.66861
Epoch 261/300
2/2 [==============================] - 1s 627ms/step - loss: 147.0620 - Si-sdr: 19.8635 - val_loss: 144.2795 - val_Si-sdr: 20.4416

Epoch 00261: val_loss did not improve from 143.66861
Epoch 262/300
2/2 [==============================] - 1s 711ms/step - loss: 144.5047 - Si-sdr: 20.2767 - val_loss: 142.7053 - val_Si-sdr: 20.5460

Epoch 00262: val_loss improved from 143.66861 to 142.70529, saving model to ./CKPT\CKP_ep_262__loss_142.70529_.h5
Epoch 263/300
2/2 [==============================] - 1s 606ms/step - loss: 145.1328 - Si-sdr: 20.2379 - val_loss: 147.5621 - val_Si-sdr: 19.8593

Epoch 00263: val_loss did not improve from 142.70529
Epoch 264/300
2/2 [==============================] - 1s 643ms/step - loss: 146.2952 - Si-sdr: 20.1618 - val_loss: 146.3513 - val_Si-sdr: 19.9954

Epoch 00264

## 2.2. Encoder 부르는 방법, Decoder에 값 넣는 방법

In [16]:
latent_size = 512
epoch = 200
BATCH_SIZE = 2

strategy = tf.distribute.MirroredStrategy(['cpu:0'])
print('장치의 수: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    model_path = './CKPT/CKP_ep_262__loss_142.70529_.h5'
    
    vq_vae = Vq_vae(latent_size, gumbel_hard=True)
    vq_vae(0, True)
    vq_vae.summary()
    
    vq_vae.load_weights(model_path)
    
    # 이렇게 하면, transforer의 input으로 들어가는 one-hot 형식의 값을 얻을 수 있음
    for inputs, label in train_dataset:
        encode = vq_vae.encoder(inputs).numpy()
        gumbelmax = vq_vae.gumbel(encode).numpy()
    
    # 이렇게 하면, transformer의 output을 vq-vae의 decoder 입력으로 넣을 수 있음
    for inputs, label in train_dataset:
        encode = vq_vae.encoder(inputs).numpy()
        gumbelmax = vq_vae.gumbel(encode).numpy()
        
        # 이렇게 이전 layer의 출렫을 넣으면 됨
        decode = vq_vae.decoder(gumbelmax).numpy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
장치의 수: 1
Model: "vqvae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
softmax_2 (Softmax)          (None, None, 512)         0         
_________________________________________________________________
encoder (Encoder)            (None, None, 512)         517248    
_________________________________________________________________
decoder (Decoder)            (None, None, 1)           516737    
_________________________________________________________________
gumbel_softmax (GumbelSoftma (None, None, 512)         0         
Total params: 1,033,985
Trainable params: 1,033,985
Non-trainable params: 0
_________________________________________________________________


# 3. Test Model

In [17]:
def mkdir_p(path):
    """ Creates a path recursively without throwing an error if it already exists
    :param path: path to create
    :return: None
    """
    if not os.path.exists(path):
        os.makedirs(path)

In [18]:
mkdir_p('./test_wav/') # Result wav 폴더 만드는 코드

In [19]:
def audiowrite(data, path, samplerate=16000, normalize=False, threaded=True):
    """ Write the audio data ``data`` to the wav file ``path``
    The file can be written in a threaded mode. In this case, the writing
    process will be started at a separate thread. Consequently, the file will
    not be written when this function exits.
    :param data: A numpy array with the audio data
    :param path: The wav file the data should be written to
    :param samplerate: Samplerate of the audio data
    :param normalize: Normalize the audio first so that the values are within
        the range of [INTMIN, INTMAX]. E.g. no clipping occurs
    :param threaded: If true, the write process will be started as a separate
        thread
    :return: The number of clipped samples
    """
    data = data.copy()
    int16_max = np.iinfo(np.int16).max
    int16_min = np.iinfo(np.int16).min

    if normalize:
        if not data.dtype.kind == 'f':
            data = data.astype(np.float)
        data /= np.max(np.abs(data))

    if data.dtype.kind == 'f':
        data *= int16_max

    sample_to_clip = np.sum(data > int16_max)
    if sample_to_clip > 0:
        print('Warning, clipping {} samples'.format(sample_to_clip))
    data = np.clip(data, int16_min, int16_max)
    data = data.astype(np.int16)

    if threaded:
        threading.Thread(target=wav_write, args=(path, samplerate, data)).start()
    else:
        wav_write(path, samplerate, data)

    return sample_to_clip

In [20]:
with tf.device('/cpu:0'):
    latent_size = 512
    sample_rate = 8000
    model_path = './CKPT/CKP_ep_262__loss_142.70529_.h5'
    
    vq_vae = Vq_vae(latent_size)
    vq_vae(0, True)
    vq_vae.summary()
    vq_vae.load_weights(model_path)

    for batch in test_dataset:
        input_batch, length_batch, name = batch

        result = vq_vae.predict(input_batch)
        
        wav_name = './test_wav/' + name[0][:-5] + '_s1.wav'
        audiowrite(result[0], wav_name, sample_rate, True, True)

Model: "vqvae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
softmax_3 (Softmax)          (None, None, 512)         0         
_________________________________________________________________
encoder (Encoder)            (None, None, 512)         517248    
_________________________________________________________________
decoder (Decoder)            (None, None, 1)           516737    
_________________________________________________________________
gumbel_softmax (GumbelSoftma (None, None, 512)         0         
Total params: 1,033,985
Trainable params: 1,033,985
Non-trainable params: 0
_________________________________________________________________


# 여기 밑에는 연습장임

In [27]:
dummy = np.random.rand(2, 10, 1)

In [72]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding

model = Sequential()
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(None, 1)))
model.add(layers.Conv1D(filters=4, kernel_size=3, padding='same'))

input_array = np.random.randn(2, 10, 1)
input_array2 = np.random.randn(2, 9, 1)
with tf.device('/cpu:0'):
    model.compile('rmsprop', 'mse')

    output_array = model.predict(input_array)
    output_array2 = model.predict(input_array2)

In [73]:
print(output_array)
print(output_array.shape)
print(output_array2.shape)

[[[-0.14930329 -0.05488385 -0.03165275  0.02511186]
  [-0.04538564 -0.02340828 -0.13736942  0.03185076]
  [-0.18286747 -0.01420305  0.02583414  0.20739384]
  [-0.15128048 -0.15295134  0.00826486 -0.03651741]
  [-0.19406578 -0.14820886 -0.24136184  0.01954714]
  [-0.11608941  0.05977409 -0.05805521  0.08262399]
  [-0.10437049  0.00281959  0.00744648  0.06939161]
  [-0.32889026 -0.11109954 -0.06513101  0.02347144]
  [-0.10914627  0.04005037 -0.11194003  0.34588984]
  [-0.55321944 -0.24241613 -0.17234027  0.2255739 ]]

 [[-0.12665638 -0.02560038 -0.07357763 -0.03386865]
  [-0.00296582  0.05382628 -0.02431939 -0.00124017]
  [-0.152773    0.01729362 -0.02049777  0.00555693]
  [-0.01630396 -0.0508009   0.00338947 -0.00536075]
  [-0.11710438 -0.0411608   0.01888775  0.10048383]
  [-0.1000828   0.06160894 -0.04584875  0.14583868]
  [-0.29426795 -0.04820506 -0.1607864   0.03541403]
  [-0.14156486  0.07812893 -0.13114211  0.12421213]
  [-0.20461929  0.08133916 -0.08211927  0.03100684]
  [-0.0094

In [358]:
batch_size = tf.shape(output_array)[0]
array1_size = tf.shape(output_array)[1]
array2_size = tf.shape(output_array2)[1]
feature_size = tf.shape(output_array)[-1]

if array1_size < array2_size:
#     append_size = array1_size - array2_size
#     append_zeros = tf.zeros([batch_size, append_size, feature_size])
#     append_zeros = tf.Variable(initial_value=tf.zeros((batch_size, append_size, feature_size)))
#     output_array2 = tf.concat([output_array2, append_zeros], axis=1)
    output_array2 = tf.slice(output_array2, [0, 0, 0], [-1, array1_size, -1])
elif array1_size > array2_size:
#     append_size = array2_size - array1_size
#     append_zeros = tf.zeros([batch_size, append_size, feature_size])
#     append_zeros = tf.Variable(initial_value=tf.zeros((batch_size, append_size, feature_size)))
#     output_array = tf.concat([output_array, append_zeros], axis=1)
    output_array = tf.slice(output_array, [0, 0, 0], [-1, array2_size, -1])

print(output_array.shape)
print(output_array2.shape)
# output_array0 = output_array[1]
# output_array20 = output_array2[1]
# target = np.sum(output_array20 * output_array0) * output_array0 / np.square(np.linalg.norm(output_array0, ord=2))
# noise = output_array20 - target
# npnp = 10 * np.log10(np.square(np.linalg.norm(target, ord=2)) / np.square(np.linalg.norm(noise, ord=2)))
# print(npnp)

# target = tf.linalg.matmul(output_array2, output_array, transpose_a=True) * output_array / tf.expand_dims(tf.experimental.numpy.square(tf.norm(output_array, axis=1)), axis=-1)
# noise = output_array2 - target
# si_sdr = 10 * tf.experimental.numpy.log10(tf.experimental.numpy.square(tf.norm(target, axis=1)) / tf.experimental.numpy.square(tf.norm(noise, axis=1)))
# si_sdr = tf.reduce_mean(si_sdr)
# print(si_sdr)

(2, 9, 1)
(2, 9, 1)


In [79]:
tf.cast(tf.equal(output_array, tf.math.reduce_max(output_array, 2, keepdims=True)), output_array.dtype)

<tf.Tensor: shape=(2, 10, 4), dtype=float32, numpy=
array([[[0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)>

In [84]:
output_array

array([[[-0.14930329, -0.05488385, -0.03165275,  0.02511186],
        [-0.04538564, -0.02340828, -0.13736942,  0.03185076],
        [-0.18286747, -0.01420305,  0.02583414,  0.20739384],
        [-0.15128048, -0.15295134,  0.00826486, -0.03651741],
        [-0.19406578, -0.14820886, -0.24136184,  0.01954714],
        [-0.11608941,  0.05977409, -0.05805521,  0.08262399],
        [-0.10437049,  0.00281959,  0.00744648,  0.06939161],
        [-0.32889026, -0.11109954, -0.06513101,  0.02347144],
        [-0.10914627,  0.04005037, -0.11194003,  0.34588984],
        [-0.55321944, -0.24241613, -0.17234027,  0.2255739 ]],

       [[-0.12665638, -0.02560038, -0.07357763, -0.03386865],
        [-0.00296582,  0.05382628, -0.02431939, -0.00124017],
        [-0.152773  ,  0.01729362, -0.02049777,  0.00555693],
        [-0.01630396, -0.0508009 ,  0.00338947, -0.00536075],
        [-0.11710438, -0.0411608 ,  0.01888775,  0.10048383],
        [-0.1000828 ,  0.06160894, -0.04584875,  0.14583868],
      

In [67]:
output_softmax = tf.nn.softmax(output_array)
output_softmax

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[0.24628553, 0.282701  , 0.2385324 , 0.23248109],
        [0.2298986 , 0.23856457, 0.25392184, 0.27761498],
        [0.2101039 , 0.2444843 , 0.26824066, 0.27717113],
        [0.22202027, 0.29728216, 0.2501223 , 0.23057525],
        [0.24084595, 0.2724257 , 0.25000373, 0.23672463]],

       [[0.23988546, 0.27957046, 0.23915865, 0.24138539],
        [0.24975686, 0.27135593, 0.24479878, 0.23408844],
        [0.24278015, 0.26340333, 0.24202275, 0.25179377],
        [0.23139507, 0.262904  , 0.25970972, 0.2459912 ],
        [0.2601803 , 0.25627998, 0.24524413, 0.23829558]]], dtype=float32)>

In [73]:
output_reshape = tf.reshape(output_softmax, [-1, 4])
output_reshape.shape

TensorShape([10, 4])

In [83]:
tf.reshape(tf.nn.softmax(output_array), [-1, 5, 4])

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[0.24628553, 0.282701  , 0.2385324 , 0.23248109],
        [0.2298986 , 0.23856457, 0.25392184, 0.27761498],
        [0.2101039 , 0.2444843 , 0.26824066, 0.27717113],
        [0.22202027, 0.29728216, 0.2501223 , 0.23057525],
        [0.24084595, 0.2724257 , 0.25000373, 0.23672463]],

       [[0.23988546, 0.27957046, 0.23915865, 0.24138539],
        [0.24975686, 0.27135593, 0.24479878, 0.23408844],
        [0.24278015, 0.26340333, 0.24202275, 0.25179377],
        [0.23139507, 0.262904  , 0.25970972, 0.2459912 ],
        [0.2601803 , 0.25627998, 0.24524413, 0.23829558]]], dtype=float32)>

In [76]:
# tf.cast(tf.equal(y, tf.reduce_max(y,1,keep_dims=True)), y.dtype)
output_hard = tf.cast(tf.equal(output_reshape, tf.math.reduce_max(output_reshape, 1, keepdims=True)), output_softmax.dtype)
output_hard

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)>

In [81]:
tf.reshape(output_hard, [-1, 5, 4])

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]]], dtype=float32)>

In [322]:
tf.transpose(output_array, perm=[0, 2, 1]).shape

TensorShape([2, 4, 5])

In [316]:
layers.Softmax(output_array)

In [317]:
np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]).shape

(3, 3)

In [65]:
np.log(10)

2.302585092994046